    L8 隐马尔可夫模型（HMM）

首先，对HMM的要素进行一些定义：

I：状态序列 [1xT]

O：观测序列 [1xT]

A：状态转移概率矩阵（转移矩阵）[NxN]

B：观测概率矩阵（隐状态）[NxM]

π：初始状态概率向量 [1xN]

In [1]:
import numpy as np

1、生成观测序列

输入: 隐马尔可夫模型 (A, B, $\pi$)

输出: 观测序列 O

In [2]:
def randomDraw(probs):
    
    #按给定概率向量随机抽取一个位置的数
    
    return np.where(np.random.multinomial(1, probs) == 1)[0][0]

In [3]:
randomDraw([0.1, 0.1, 0.9])

2

In [4]:
def get_obsSquence(A, B, π, T):
    
    # T: 观测序列长度
    
    obsSequence = np.zeros(T, dtype = int)
    staSequence = np.zeros(T, dtype = int)
    
    staSequence[0] = randomDraw(π)
    obsSequence[0] = randomDraw(B[staSequence[0], :])
    
    for t in range(1, T):
        staSequence[t] = randomDraw(A[staSequence[t-1], :])
        obsSequence[t] = randomDraw(B[staSequence[t], :])
    
    return staSequence, obsSequence
    

In [5]:
probs = [1/4]*4
trans_mat = np.array([[0, 1, 0, 0], [0.4, 0, 0.6, 0], [0, 0.4, 0, 0.6], [0, 0, 0.5, 0.5]])
prob_mat = np.array([[0.5, 0.5], [0.3, 0.7], [0.6, 0.4], [0.8, 0.2]])

In [6]:
color = ['红色', '白色']

In [7]:
state, observe = get_obsSquence(trans_mat, prob_mat, probs, 6)

In [8]:
[color[i] for i in observe]

['红色', '红色', '白色', '红色', '白色', '红色']

2、给定HMM模型和观测序列O，求O出现的概率

输入：HMM模型，观测序列O

输出：观测序列出现的概率 P(O|$\lambda$)

    1、前向算法

In [9]:
def forwardAlgo(A, B, π, O):
    
    N = A.shape[0]
    T = len(O)
    
    P = np.zeros((T, N))
    
    P[0] = π * B[:, O[0]]
    
    for t in range(1, T):
        P[t] = np.dot(P[t-1], A) * B[:, observe[t]] 
    
    return P , np.sum(P[T-1])

In [10]:
forwardAlgo(trans_mat, prob_mat, probs, observe)

(array([[0.125     , 0.075     , 0.15      , 0.2       ],
        [0.015     , 0.0555    , 0.087     , 0.152     ],
        [0.0111    , 0.03486   , 0.04372   , 0.02564   ],
        [0.006972  , 0.0085764 , 0.0202416 , 0.0312416 ],
        [0.00171528, 0.01054805, 0.00830666, 0.00555315],
        [0.00210961, 0.00151138, 0.00546324, 0.00620846]]),
 0.015292690880000005)

    2、后向算法

In [11]:
def backwardAlgo(A, B, π, O):
    
    N = A.shape[0]
    T = len(O)
    P = np.zeros((T, N))
    
    P[T-1] = 1
    
    for t in reversed(range(T-1)):
        P[t] = np.dot(A, B[:, O[t+1]]) * P[t+1]
    p = np.dot(P[0] * π, B[:, O[0]])
    
    return P, p

In [12]:
backwardAlgo(trans_mat, prob_mat, probs, observe)

(array([[0.01323   , 0.03399926, 0.03456   , 0.03087   ],
        [0.0441    , 0.06071296, 0.0576    , 0.0441    ],
        [0.063     , 0.137984  , 0.144     , 0.147     ],
        [0.21      , 0.2464    , 0.24      , 0.21      ],
        [0.3       , 0.56      , 0.6       , 0.7       ],
        [1.        , 1.        , 1.        , 1.        ]]),
 0.01556169432)

3、无监督学习算法

给定观测序列O，求HMM模型

Baum-Welch算法

In [13]:
def get_gamma(α, β, N, T):
    
    #计算γ=[TxN]
    
    γ = np.zeros((T, N))
    
    for t in range(T):
        γ[t] = α[t] * β[t] / np.dot(α[t], β[t])
    
    return γ

In [22]:
def get_xi(A, B, α, β, N, T):
    
    #计算ξ=[TxNxN]
    
    ξ = np.zeros((T-1, N, N))
    
    for t in range(T-1):
        ξ[t] = (α[t] * (A * B[:, observe[t+1]] * β[t+1]).T).T \
               /np.sum(α[t] * (A * B[:, observe[t+1]] * β[t+1]).T)
    
    return ξ

In [25]:
def Baum_Welch_Train(Trans_mat, Prob_mat, Init_mat, O, N, M, ϵ):
    
    #初始化HMM，但是取均值为初始值并不是很好
    # π = np.ones(N)/N
    # A = np.ones((N, N))/N
    # B = np.ones((N, M))/M 
    
    A = np.copy(Trans_mat)
    B = np.copy(Prob_mat)
    π = np.copy(Init_mat)
    
    T = len(O)
    
    step = 1
    
    while 1:
        
        Alpha, p1 = forwardAlgo(A, B, π, O) #Alpha=[TxN]
        Beta, p2 = backwardAlgo(A, B, π, O) #Beta=[TxN]
    
        γ = get_gamma(Alpha, Beta, N, T)
        ξ = get_xi(A, B, Alpha, Beta, N, T)
        
        π_new = γ[0]
        A_new = (np.sum(ξ, 0).T/np.sum(γ[:T-1, :], 0)).T
        B_new = np.copy(B)

        for i in range(M):
            B_new[:,i] = np.sum(γ[O==i, :], 0) / np.sum(γ, 0)
        
        step += 1
        
        if np.max(abs(π - π_new)) < ϵ and \
           np.max(abs(A - A_new)) < ϵ and \
           np.max(abs(B - B_new)) < ϵ:
            break
        if step >= 10:
            break
            
        π[:] = π_new
        A[:] = A_new
        B[:] = B_new
        
    return π, A, B

In [26]:
Baum_Welch_Train(trans_mat, prob_mat, probs, observe, 4, 2, 0.1)

(array([3.46150120e-06, 2.26361963e-06, 1.53503783e-02, 9.84643897e-01]),
 array([[0.00000000e+00, 1.98870934e+04, 0.00000000e+00, 0.00000000e+00],
        [4.49400577e-05, 0.00000000e+00, 9.98059981e-01, 0.00000000e+00],
        [0.00000000e+00, 6.99486225e-02, 0.00000000e+00, 9.19135670e-01],
        [0.00000000e+00, 0.00000000e+00, 1.65583964e-01, 4.78392236e-01]]),
 array([[6.88781642e-02, 9.31121836e-01],
        [1.00000000e+00, 1.30263744e-12],
        [2.20834941e-01, 7.79165059e-01],
        [7.58698199e-01, 2.41301801e-01]]))

4、预测算法 

Viterbi算法

输入：HMM模型，观测序列O

输出：最优路径 I

In [27]:
δ = np.zeros((6, 4))

In [30]:
δ[0] = probs * prob_mat[:, observe[0]]

In [58]:
δ[-1]

array([0., 0., 0., 0.])

In [32]:
A = np.copy(trans_mat)

In [34]:
A.

array([[0. , 1. , 0. , 0. ],
       [0.4, 0. , 0.6, 0. ],
       [0. , 0.4, 0. , 0.6],
       [0. , 0. , 0.5, 0.5]])

In [39]:
δ[0]*A.T

array([[0.   , 0.03 , 0.   , 0.   ],
       [0.125, 0.   , 0.06 , 0.   ],
       [0.   , 0.045, 0.   , 0.1  ],
       [0.   , 0.   , 0.09 , 0.1  ]])

In [44]:
np.argmax(δ[0]*A.T, 1)

array([1, 0, 3, 3])

In [86]:
def Viterbi_Algo(trans_mat, prob_mat, init_mat, O):
    
    A = np.copy(trans_mat)
    B = np.copy(prob_mat)
    π = np.copy(init_mat)
    
    N = A.shape[0]
    T = len(O)
    
    δ = np.zeros((T, N))
    Ψ = np.zeros((T, N), dtype = int)
    
    δ[0] = π * B[:, O[0]]
    
    for t in range(1, T):
        δ[t] = np.max(δ[t-1] * A.T, 1) * B[:, O[t]]
        Ψ[t] = np.argmax(δ[t-1] * A.T, 1)
        
    return δ, Ψ

In [97]:
def Vitetbi_path(δ, Ψ):
    
    #推最优路径
    
    T = δ.shape[0]
    i = np.zeros(T, dtype = int)
    P = np.max(δ[-1])
    i[-1] = np.argmax(δ[-1])
    
    for j in reversed(range(T-1)):
        i[j] = Ψ[j+1, i[j+1]]
    
    return i 

In [87]:
mat1, mat2 = Viterbi_Algo(trans_mat, prob_mat, probs, observe)

In [99]:
Vitetbi_path(mat1, mat2)

array([3, 2, 1, 0, 1, 2])